In [27]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [28]:
churnData = pd.read_csv('Customer-Churn.csv')
churnData.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [29]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [30]:
churnData["TotalCharges"] = pd.to_numeric(churnData["TotalCharges"], errors = 'coerce')   #errors = coerce important cause i trnsform the non numeric values into Nan

#churnData['TotalCharges'] = churnData['TotalCharges'].astype(float)

churnData["TotalCharges"].isna().sum()

#have 11 NaN

11

In [31]:
#Check null values in the entire dataframe

churnData.isna().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [32]:
#replace NaN 
churnData["TotalCharges"] = churnData["TotalCharges"].fillna(churnData["TotalCharges"].mean())



In [33]:
toscale = churnData[["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]]

In [34]:
toscale.head()
churnData["Churn"].value_counts()


#unbalanced dataset

No     5174
Yes    1869
Name: Churn, dtype: int64

In [35]:
def encode_churn(x):
    if x == 'No':
        return 0
    else:
        return 1
churnData["Churn"] = churnData["Churn"].apply(encode_churn)

#encoding Churn

churnData["Churn"].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [36]:
#function doesnt work : one hot encoding
#from sklearn.preprocessing import OneHotEncoder
#onehot_encoder = OneHotEncoder(sparse=False)
#onehot_encoded = onehot_encoder.fit_transform(y)



In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform(toscale)

X_scaled = pd.DataFrame(data = scaled, columns = toscale.columns)
X = X_scaled 

y = churnData["Churn"]
y

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: int64

In [38]:
#split the data 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [39]:
#Logistic regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test,y_test)


0.7830777967064169

In [40]:
#Balancing the data 

churnData["Churn"].value_counts()     #majority class is 0, no churn

0    5174
1    1869
Name: Churn, dtype: int64

In [41]:
#Undersampling

from sklearn.utils import resample

churn_yes = churnData[churnData["Churn"] == 1]
churn_no = churnData[churnData["Churn"] == 0]



In [42]:
churn_no_undersampled = resample(churn_no, replace=False, n_samples = len(churn_yes))



data_downsampled = pd.concat([churn_no_undersampled, churn_yes], axis=0)
data_downsampled.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
1835,Female,0,Yes,No,1,Yes,No,No,No,No,No,No,Month-to-month,45.00,45.00,0
4963,Male,0,Yes,No,47,Yes,No,Yes,Yes,No,Yes,Yes,Two year,73.60,3522.65,0
5325,Male,0,Yes,Yes,5,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,20.55,99.45,0
3086,Male,0,No,No,17,Yes,No,No,Yes,Yes,No,Yes,Month-to-month,95.65,1640.00,0
7042,Male,0,No,No,66,Yes,Yes,No,Yes,Yes,Yes,Yes,Two year,105.65,6844.50,0


In [43]:
print(len(churn_no_undersampled))
print(len(churn_yes))

1869
1869


In [44]:
#Refit the model with the new dataset


X_downsampled = data_downsampled[["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]]
y_downsampled = data_downsampled[["Churn"]]


In [45]:
#split the data 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_downsampled, y_downsampled, random_state=0)

In [46]:
#Logistic regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test,y_test)

#Model accuracy is worst after the downsampling, lets try oversampling

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.732620320855615

In [51]:
#OVERSAMPLING

churn_yes_oversampled = resample(churn_yes, replace=True, n_samples = len(churn_no))

print(len(churn_yes_oversampled))
print(len(churn_no))

data_oversampled = pd.concat([churn_yes_oversampled, churn_no], axis=0)
data_oversampled.head()

5174
5174


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
1223,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,75.90,143.35,1
842,Female,0,Yes,Yes,12,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,19.80,229.60,1
6088,Female,0,No,No,1,Yes,No,No,No,No,No,Yes,Month-to-month,55.40,55.40,1
6708,Female,1,No,No,14,Yes,No,No,No,No,No,No,Month-to-month,74.30,1096.25,1
5944,Male,0,No,No,12,Yes,Yes,No,No,Yes,No,No,Month-to-month,56.65,654.85,1


In [52]:
X_oversampled = data_oversampled[["tenure", "SeniorCitizen", "MonthlyCharges", "TotalCharges"]]
y_oversampled = data_oversampled[["Churn"]]

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_oversampled, y_oversampled, random_state=0)

In [54]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
model.score(X_test,y_test)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7270970235794356

IMBALANCED   0.78

UNDERSAMPLED 0.73

OVERSAMPLED  0.727